# Test Suites and Reports for Bicycle Demand Prediction

In [3]:
import pandas as pd
import numpy as np
import requests
import zipfile
import io

from datetime import datetime, time
from sklearn import datasets, ensemble

from evidently import ColumnMapping

from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, RegressionPreset

from evidently.test_suite import TestSuite
from evidently.test_preset import DataDriftTestPreset, DataQualityTestPreset, RegressionTestPreset
from evidently.tests import TestValueMeanError, TestValueMAE, TestColumnDrift, TestColumnValueMean, TestColumnQuantile

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Bicycle Demand Data

In [4]:
content = requests.get("https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip").content
with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(arc.open("hour.csv"), header=0, sep=',', parse_dates=['dteday'], index_col='dteday')

In [5]:
raw_data.index = raw_data.apply(
    lambda row: datetime.combine(row.name, time(hour=int(row['hr']))), axis = 1)

In [6]:
raw_data.head()

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
2011-01-01 00:00:00,1,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
2011-01-01 01:00:00,2,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2011-01-01 02:00:00,3,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
2011-01-01 03:00:00,4,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
2011-01-01 04:00:00,5,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## Regression Model

### Model training

In [ ]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday']

In [ ]:
reference = raw_data.loc['2011-01-01 00:00:00':'2011-01-28 23:00:00']
current = raw_data.loc['2011-01-29 00:00:00':'2011-02-28 23:00:00']

In [ ]:
reference.head()

In [ ]:
regressor = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)

In [ ]:
regressor.fit(reference[numerical_features + categorical_features], reference[target])

In [ ]:
ref_prediction = regressor.predict(reference[numerical_features + categorical_features])
current_prediction = regressor.predict(current[numerical_features + categorical_features])

In [ ]:
reference['prediction'] = ref_prediction
current['prediction'] = current_prediction

In [ ]:
reference.head()

## Test Presets

In [ ]:
column_mapping = ColumnMapping()

column_mapping.target = target
column_mapping.prediction = prediction
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

In [ ]:
data_quality_test = TestSuite(tests=[
    DataQualityTestPreset(),
])

data_quality_test.run(reference_data=reference, current_data=current, column_mapping=column_mapping)
data_quality_test.show(mode='inline')

## Reference-based conditions

In [ ]:
data_quality_test = TestSuite(tests=[
    TestColumnDrift(column_name='temp'),
    TestColumnValueMean(column_name='temp'),
    TestColumnQuantile(column_name='temp', quantile=0.5)
])

data_quality_test.run(reference_data=reference, current_data=current, column_mapping=column_mapping)

data_quality_test.show(mode='inline')

## Custom conditions

In [ ]:
data_quality_test = TestSuite(tests=[
    TestColumnValueMean(column_name='temp', lt=0.3, gt=0.2),
    TestColumnQuantile(column_name='temp', quantile=0.5, gt=0.2 )
])

data_quality_test.run(reference_data=None, current_data=current, column_mapping=column_mapping)

data_quality_test.show(mode='inline')

## Output Formats

In [ ]:
data_quality_test.json()

In [ ]:
data_quality_test.save_html('test_suite.html')

In [ ]:
data_quality_test.save_json('test_suite.json')

In [ ]:
data_quality_test._save('snapshot.json')

In [ ]:
loaded_test_suite = TestSuite._load('snapshot.json')

In [ ]:
loaded_test_suite.show(mode='inline')

In [ ]:
loader_test_suite = TestSuite._load('snapshot.json')

In [ ]:
loader_test_suite.show(mode='inline')

## Reports

In [ ]:
regression_report = Report(
    metrics=[
        RegressionPreset()
    ]
)

regression_report.run(reference_data=reference, current_data=current, column_mapping=column_mapping)

regression_report.show(mode='inline')

In [ ]:
drift_report = Report(
    metrics=[
        DataDriftPreset()
    ]
)

drift_report.run(reference_data=reference, current_data=current, column_mapping=column_mapping)

drift_report.show(mode='inline')

# Support Evidently
Enjoyed the tutorial? Star Evidently on GitHub to contribute back! This helps us continue creating free open-source tools for the community. https://github.com/evidentlyai/evidently